### Importing libraries

In [1]:
import anuvada
import numpy as np
import torch
import pandas as pd

In [2]:
from anuvada.models.classification_cnn import ClassificationCNN

### Creating the dataset

In [3]:
from anuvada.datasets.data_loader import CreateDataset
from anuvada.datasets.data_loader import LoadData

In [4]:
data = CreateDataset()

In [5]:
df = pd.read_csv('/home/sandeep/Downloads/movie_summary_filtered.csv',encoding='utf-8')

In [6]:
df.head()

,ID,Name,Genre,summary
0,975900,Ghosts of Mars,Thriller,"Set in the second half of the 22nd century, th..."
1,9363483,White Of The Eye,Thriller,A series of murders of rich young women throug...
2,261236,A Woman in Flames,Drama,"Eva, an upper class housewife, becomes frustra..."
3,18998739,The Sorcerer's Apprentice,Family Film,"Every hundred years, the evil Morgana returns..."
4,6631279,Little city,Romantic comedy,"Adam, a San Francisco-based artist who works a..."


In [7]:
# passing only the first 512 samples, don't have a GPU here!
y = list(df.Genre.values)[0:512]
x = list(df.summary.values)[0:512]

In case of multilabel classification, the labels are expected to be separated by '__'.

In [8]:
x, y, token2id, label2id, lengths_mask = data.create_dataset(x,y, folder_path='test', max_doc_tokens=500,word2vec=True,multilabel=False)

  0%|          | 2/512 [00:00<00:32, 15.78it/s]

Building vocabulary...


100%|██████████| 512/512 [00:12<00:00, 39.44it/s]


Bulding Word2vec model...
Word2vec model saved...
Building dataset...
Datasets saved in folder test


### Loading created dataset

In [9]:
l = LoadData()

In [10]:
x, y, token2id, label2id, lengths_mask = l.load_data_from_path('test')

In [11]:
id2token = {v: k for k, v in token2id.iteritems()}

### Change into torch vectors

In [12]:
x = torch.from_numpy(x)

In [13]:
y = torch.from_numpy(np.array(y))

In [14]:
len(id2token)

3818

### Create attention classifier

You have to pass multilabel loss function as a parameter.

In [16]:
acf = ClassificationCNN(vocab_size=len(token2id),embed_dim=300,num_classes=len(label2id),word2vec_path='test/word2vec_300_5_5')

In [17]:
loss = acf.fit(x,y,batch_size=64,epochs=2,validation_split=0.2)

Epoch 1 / 2
[========================================] 100%	loss: 3.9326loss: 3.9326    val_loss: 3.8769

Epoch 2 / 2
[========================================] 100%	loss: 3.6876loss: 3.6876    val_loss: 3.7266



### Predicting

Prediction needs more than one minibatch at this moment.

In [18]:
acf.predict(x[0:128])


-0.3291 -0.2909 -0.2035  ...   0.0128 -0.2124  0.0561
-0.3501 -0.2921 -0.2109  ...   0.0138 -0.2259  0.0496
-0.3315 -0.2884 -0.1988  ...   0.0129 -0.2124  0.0578
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 128x53]